# Word Vectorization - Lab

## Introduction

In this lab, we'll learn how tokenize and vectorize text documents, create an use a Bag of Words, and identify words unique to individual documents using TF-IDF Vectorization. 

## Objectives

You will be able to: 

* Tokenize a corpus of words and identify the different choices to be made while parsing them
* Use a Count Vectorization strategy to create a Bag of Words
* Use TF-IDF Vectorization with multiple documents to identify words that are important/unique to certain documents

## Let's get started!

Run the cell below to import everything necessary for this lab.  

In [ ]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
np.random.seed(0)

### Our Corpus

In this lab, we'll be working with 20 different documents, each containing song lyrics from either Garth Brooks or Kendrick Lamar albums.  

The songs are contained within the `data` subdirectory, contained within the same folder as this lab.  Each song is stored in a single file, with files ranging from `song1.txt` to `song20.txt`.  

To make it easy to read in all of the documents, use a list comprehension to create a list containing the name of every single song file in the cell below. 

In [3]:
filenames = ['song' + str(i) + '.txt' for i in range(1, 21)]

Next, let's import a single song to see what our text looks like so that we can make sure we clean and tokenize it correctly. 

In the cell below, read in and print out the lyrics from `song11.txt`.  Use vanilla python, no pandas needed.  

In [4]:
with open('data/song11.txt') as f:
    lyric = f.readlines()
    print(lyric)

lyric[1]

['[Kendrick Lamar:]\n', "Love, let's talk about love\n", 'Is it anything and everything you hoped for?\n', 'Or do the feeling haunt you?\n', 'I know the feeling haunt you\n', '[SZA:]\n', 'This may be the night that my dreams might let me know\n', 'All the stars approach you, all the stars approach you, all the stars approach you\n', 'This may be the night that my dreams might let me know\n', 'All the stars are closer, all the stars are closer, all the stars are closer\n', '[Kendrick Lamar:]\n', "Tell me what you gon' do to me\n", "Confrontation ain't nothin' new to me\n", 'You can bring a bullet, bring a sword, bring a morgue\n', "But you can't bring the truth to me\n", 'Fuck you and all your expectations\n', "I don't even want your congratulations\n", 'I recognize your false confidence\n', 'And calculated promises all in your conversation\n', 'I hate people that feel entitled\n', "Look at me crazy 'cause I ain't invite you\n", 'Oh, you important?\n', "You the moral to the story? You e

"Love, let's talk about love\n"

### Tokenizing our Data

Before we can create a Bag of Words or vectorize each document, we need to clean it up and split each song into an array of individual words.  Computers are very particular about strings. If we tokenized our data in it's current state, we would run into the following problems:

1. Counting things that aren't actually words.  In the example above, `"[Kendrick]"` is a note specifying who is speaking, not a lyric contained in the actual song, so it should be removed.  
1. Punctuation and capitalization would mess up our word counts.  To the python interpreter, `love`, `Love`, `Love?`, and `Love\n` are all unique words, and would all be counted separately.  We need to remove punctuation and capitalization, so that all words will be counted correctly. 

Consider the following sentences from the example above:

`"Love, let's talk about love\n", 'Is it anything and everything you hoped for?\n'`

After tokenization, this should look like:

`['love', 'let's', 'talk', 'about', 'love', 'is', 'it', 'anything', 'and', 'everything', 'you', 'hoped', 'for']`

Tokenization is pretty tedious if we handle it manually, and would probably make use of Regular Expressions, which is outside the scope of this lab.  In order to keep this lab moving, we'll use a library function to clean and tokenize our data so that we can move onto vectorization.  

Tokenization is a required task for just about any Natural Language Processing (NLP) task, so great industry-standard tools exist to tokenize things for us, so that we can spend our time on more important tasks without getting bogged down hunting every special symbol or punctuation in a massive dataset. For this lab, we'll make use of the tokenizer in the amazing `nltk` library, which is short for _Natural Language Tool Kit_.

**_NOTE:_** NLTK requires extra installation methods to be run the first time certain methods are used.  If `nltk` throws you an error about needing to install additional packages, follow the instructions in the error message to install the dependencies, and then rerun the cell.  

Before we tokenize our songs, we'll do only a small manual bit of cleaning.  In the cell below, write a function that allows us to remove lines that have `['artist names']` in it, to ensure that our song files contain only lyrics that are actually in the song. For the lines that remain, make every word lowercase, remove newline characters `\n`, and any of the following punctuation marks: `",.'?!"`

Test the function on `test_song` to show that it has successfully removed `'[Kendrick Lamar:]'` and other instances of artist names from the song and returned it.  

In [ ]:
test = ["Love, let's talk about love\n"]

In [5]:
def clean_song(song):
    new_song =[]
    for item in song:
        if item.startswith('[') == False:
            for symbol in ",.''?!\n":
                if symbol in item:
                    item = item.replace(symbol,'').lower()
            new_song.append(item)        
    return new_song

song_without_brackets = clean_song(lyric)
print(song_without_brackets)

['love lets talk about love', 'is it anything and everything you hoped for', 'or do the feeling haunt you', 'i know the feeling haunt you', 'this may be the night that my dreams might let me know', 'all the stars approach you all the stars approach you all the stars approach you', 'this may be the night that my dreams might let me know', 'all the stars are closer all the stars are closer all the stars are closer', 'tell me what you gon do to me', 'confrontation aint nothin new to me', 'you can bring a bullet bring a sword bring a morgue', 'but you cant bring the truth to me', 'fuck you and all your expectations', 'i dont even want your congratulations', 'i recognize your false confidence', 'and calculated promises all in your conversation', 'i hate people that feel entitled', 'look at me crazy cause i aint invite you', 'oh you important', 'you the moral to the story you endorsin', 'motherfucker i dont even like you', 'corrupt a mans heart with a gift', 'thats how you find out who you d

Great. Now, write a function that takes in songs that have had their brackets removed, joins all of the lines into a single string, and then uses `tokenize()` on it to get a fully tokenized version of the song.  Test this funtion on `song_without_brackets` to ensure that the function works. 

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Thien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def tokenize(song):
    joined = ' '.join(song)
    token = word_tokenize(joined)
    return token

tokenized_test_song = tokenize(song_without_brackets)
tokenized_test_song[:10]

['love',
 'lets',
 'talk',
 'about',
 'love',
 'is',
 'it',
 'anything',
 'and',
 'everything']

Great! Now that we know the ability to tokenize our songs, we can move onto Vectorization. 

### Count Vectorization

Machine Learning algorithms don't understand strings.  However, they do understand math, which means they understand vectors and matrices.  By **_Vectorizing_** the text, we just convert the entire text into a vector, where each element in the vector represents a different word.  The vector is the length of the entire vocabulary--usually, every word that occurs in the English language, or at least every word that appears in our corpus.  Any given sentence can then be represented as a vector where all the vector is 1 (or some other value) for each time that word appears in the sentence. 

Consider the following example: 

<center>"I scream, you scream, we all scream for ice cream."</center>

| 'aardvark' | 'apple' | [...] | 'I' | 'you' | 'scream' | 'we' | 'all' | 'for' | 'ice' | 'cream' | [...] | 'xylophone' | 'zebra' |
|:----------:|:-------:|:-----:|:---:|:-----:|:--------:|:----:|:-----:|:-----:|:-----:|:-------:|:-----:|:-----------:|:-------:|
|      0     |    0    |   0   |  1  |   1   |     3    |   1  |   1   |   1   |   1   |    1    |   0   |      0      |    0    |

This is called a **_Sparse Representation_**, since the strong majority of the columns will have a value of 0.  Note that elements corresponding to words that do not occur in the sentence have a value of 0, while words that do appear in the sentence have a value of 1 (or 1 for each time it appears in the sentence).

Alternatively, we can represent this sentence as a plain old python dictionary of word frequency counts:

```python
BoW = {
    'I':1,
    'you':1,
    'scream':3,
    'we':1,
    'all':1,
    'for':1,
    'ice':1,
    'cream':1
}
```

Both of these are examples of **_Count Vectorization_**. They allow us to represent a sentence as a vector, with each element in the vector corresponding to how many times that word is used.

#### Positional Information and Bag of Words

Notice that when we vectorize a sentence this way, we lose the order that the words were in.  This is the **_Bag of Words_** approach mentioned earlier.  Note that sentences that contain the same words will create the same vectors, even if they mean different things--e.g. `'cats are scared of dogs'` and `'dogs are scared of cats'` would both produce the exact same vector, since they contain the same words.  

In the cell below, create a function that takes in a tokenized, cleaned song and returns a Count Vectorized representation of it as a python dictionary. Add in an optional parameter called `vocab` that defaults to `None`. This way, if we are using a vocabulary that contains words not seen in the song, we can still use this function by passing it in to the `vocab` parameter. 

**_Hint:_**  Consider using a `set` object to make this easier!

In [8]:
def count_vectorize(song, vocab=None):
    if vocab:
        unique = vocab
    else:
        unique = set(song)
        
    vector = {}
    
    for item in unique:
        vector[item] = 0
    for word in song:
        vector[word] = vector.get(word) + 1
    return vector

test_vectorized = count_vectorize(tokenized_test_song)
print(test_vectorized)

{'this': 6, 'cant': 1, 'no': 4, 'stay': 1, 'do': 8, 'might': 6, 'endorsin': 1, 'cryin': 1, 'is': 3, 'wouldnt': 1, 'can': 1, 'losin': 1, 'sword': 1, 'heart': 1, 'all': 22, 'youre': 1, 'ya': 1, 'nothin': 1, 'corrupt': 1, 'at': 2, 'how': 1, 'wont': 1, 'tryna': 1, 'feel': 4, 'bullet': 1, 'give': 1, 'morgue': 1, 'may': 6, 'want': 2, 'got': 3, 'runnin': 1, 'what': 1, 'bad': 1, 'for': 7, 'mans': 1, 'get': 2, 'end': 1, 'your': 5, 'recognize': 1, 'and': 6, 'truth': 1, 'lets': 3, 'credit': 1, 'way': 2, 'my': 7, 'who': 2, 'in': 2, 'dreams': 6, 'making': 1, 'live': 1, 'everything': 3, 'dealin': 1, 'with': 3, 'expectations': 1, 'confidence': 1, 'confrontation': 1, 'moral': 1, 'feeling': 7, 'winnin': 1, 'let': 6, 'ego': 1, 'were': 1, 'anything': 3, 'move': 1, 'another': 2, 'breathin': 1, 'dont': 2, 'reason': 3, 'aint': 4, 'loved': 1, 'buildin': 1, 'time': 1, 'small': 1, 'ever': 1, 'life': 2, 'story': 1, 'on': 1, 'find': 1, 'invite': 1, 'covered': 1, 'calculated': 1, 'down': 1, 'days': 1, 'i': 15, 'r

Great! You've just successfully vectorized your first text document! Now, let's look at a more advanced type of vectorization, TF-IDF!

### TF-IDF Vectorization

TF-IDF stands for **_Term Frequency, Inverse Document Frequency_**.  This is a more advanced form of vectorization that weights each term in a document by how unique it is to the given document it is contained in, which allows us to summarize the contents of a document using a few key words.  If the word is used often in many other documents, it is not unique, and therefore probably not too useful if we wanted to figure out how this document is unique in relation to other documents.  Conversely, if a word is used many times in a document, but rarely in all the other documents we are considering, then it is likely a good indicator for telling us that this word is important to the document in question.  

The formula TF-IDF uses to determine the weights of each term in a document is **_Term Frequency_** multipled by **_Inverse Document Frequency_**, where the formula for Term Frequency is:

$$\large Term\ Frequency(t) = \frac{number\ of\ times\ t\ appears\ in\ a\ document} {total\ number\ of\ terms\ in\ the\ document} $$
<br>
<br>
Complete the following function below to calculate term frequency for every term in a document.  

In [9]:
test_vectorized

{'this': 6,
 'cant': 1,
 'no': 4,
 'stay': 1,
 'do': 8,
 'might': 6,
 'endorsin': 1,
 'cryin': 1,
 'is': 3,
 'wouldnt': 1,
 'can': 1,
 'losin': 1,
 'sword': 1,
 'heart': 1,
 'all': 22,
 'youre': 1,
 'ya': 1,
 'nothin': 1,
 'corrupt': 1,
 'at': 2,
 'how': 1,
 'wont': 1,
 'tryna': 1,
 'feel': 4,
 'bullet': 1,
 'give': 1,
 'morgue': 1,
 'may': 6,
 'want': 2,
 'got': 3,
 'runnin': 1,
 'what': 1,
 'bad': 1,
 'for': 7,
 'mans': 1,
 'get': 2,
 'end': 1,
 'your': 5,
 'recognize': 1,
 'and': 6,
 'truth': 1,
 'lets': 3,
 'credit': 1,
 'way': 2,
 'my': 7,
 'who': 2,
 'in': 2,
 'dreams': 6,
 'making': 1,
 'live': 1,
 'everything': 3,
 'dealin': 1,
 'with': 3,
 'expectations': 1,
 'confidence': 1,
 'confrontation': 1,
 'moral': 1,
 'feeling': 7,
 'winnin': 1,
 'let': 6,
 'ego': 1,
 'were': 1,
 'anything': 3,
 'move': 1,
 'another': 2,
 'breathin': 1,
 'dont': 2,
 'reason': 3,
 'aint': 4,
 'loved': 1,
 'buildin': 1,
 'time': 1,
 'small': 1,
 'ever': 1,
 'life': 2,
 'story': 1,
 'on': 1,
 'find': 1,


In [10]:
def term_frequency(BoW_dict):
    freq_dict = {}
    total = sum(BoW_dict.values())
    for key in BoW_dict:
        freq_dict[key] = BoW_dict[key]/total
    return freq_dict
    
test = term_frequency(test_vectorized)
print(list(test.values())[10:20])

[0.0020833333333333333, 0.0020833333333333333, 0.0020833333333333333, 0.0020833333333333333, 0.04583333333333333, 0.0020833333333333333, 0.0020833333333333333, 0.0020833333333333333, 0.0020833333333333333, 0.004166666666666667]


The formula for Inverse Document Frequency is:  
<br>  
<br>
$$\large  IDF(t) =  log_e(\frac{Total\ Number\ of\ Documents}{Number\ of\ Documents\ with\ t\ in\ it})$$

Now that we have this, we can easily calculate _Inverse Document Frequency_.  In the cell below, complete the following function.  this function should take in the list of dictionaries, with each item in the list being a Bag of Words representing the words in a different song. The function should return a dictionary containing the inverse document frequency values for each word.  

In [21]:
#note to self: already takes in count vectorize of each song
def inverse_document_frequency(list_of_dicts):
    num = len(list_of_dicts)
    
    t_count = {}
    for dictionary in list_of_dicts:
        for word in dictionary:
            t_count[word] = t_count.get(word,0) +1
    
    for word in t_count:
        t_count[word] = np.log(num/t_count[word])

    return t_count

#new_test = [test_vectorized, test_vectorized]  #(used for testing fx)
#inverse_document_frequency(new_test)      #(testing fx)

### Computing TF-IDF

Now that we can compute both Term Frequency and Inverse Document Frequency, computing an overall TF-IDF value is simple! All we need to do is multiply the two values.  

In the cell below, complete the `tf_idf()` function.  This function should take in a list of dictionaries, just as the `inverse_document_frequency()` function did.  This function return a new list of dictionaries, with each dictionary containing the tf-idf vectorized representation of a corresponding song document. 

**_NOTE:_** Each document should contain the full vocabulary of the entire combined corpus.  

In [33]:
def tf_idf(list_of_dicts):
    tfidf = []
    idf = inverse_document_frequency(list_of_dicts)
    
    for dictionary in list_of_dicts:
        tf = term_frequency(dictionary)
        for word in tf:
            tf[word] = tf[word]*idf[word]
        tfidf.append(tf)
    return tfidf

### Vectorizing All Documents

Now that we've created all the necessary helper functions, we can load in all of our documents and run each through the vectorization pipeline we've just created.

In the cell below, complete the `main` function.  This function should take in a list of file names (provided for you in the `filenames` list we created at the start), and then:

1. Read in each document
1. Tokenize each document
1. Convert each document to a Bag of Words (dictionary representation)
1. Return a list of dictionaries vectorized using tf-idf, where each dictionary is a vectorized representation of a document.  

**_HINT:_** Remember that all files are stored in the `data/` directory.  Be sure to append this to the filename when reading in each file, otherwise the path won't be correct!

In [ ]:
filenames

In [43]:
def main(filenames):
    post_vectorize = []
    #Read each file
    for file in filenames:
        with open('data/{}'.format(file)) as f:
            lyric = f.readlines()
    
        #tokenize +clean it
        tokenized = tokenize(clean_song(lyric))
        
        #BoW Conversion
        vectorized = count_vectorize(tokenized)
        post_vectorize.append(vectorized)
        
    #tfidf 
    return tf_idf(post_vectorize)

tfidf_all = main(filenames)
print(list(tf_idf_all_docs)[0])

{'red': 0.0057839023929447606, 'when': 0.0010874236095693062, 'japan': 0.009133330102298753, 'forget': 0.007020076503030628, 'strings': 0.009133330102298753, 'teach': 0.027399990306896257, 'all': 0.002721262820904997, 'vincent': 0.009133330102298753, 'fourteen': 0.009133330102298753, 'how': 0.0018226737827915257, 'their': 0.0057839023929447606, 'bullet': 0.007020076503030628, 'ventura': 0.009133330102298753, 'theres': 0.003200677208837432, 'said': 0.007020076503030628, 'water': 0.007020076503030628, 'runnin': 0.012679521595608754, 'such': 0.009133330102298753, 'song': 0.007020076503030628, 'her': 0.0036706487936766347, 'valentines': 0.009133330102298753, 'now': 0.004906822903762501, 'for': 0.0052534501962494436, 'raised': 0.009133330102298753, 'borderline': 0.009133330102298753, 'state': 0.009133330102298753, 'highway': 0.007020076503030628, 'green': 0.007020076503030628, 'team': 0.007020076503030628, 'queens': 0.009133330102298753, 'heard': 0.014040153006061255, 'scenes': 0.0091333301

### Visualizing our Vectorizations

Now that we have a tf-idf representation each document, we can move on to the fun part--visualizing everything!

Let's investigate how many dimensions our data currently has.  In the cell below, examine our dataset to figure out how many dimensions our dataset has. 

**_HINT_**: Remember that every word is it's own dimension!

In [55]:
num_dims = sum(len(tfidf_all[i].keys()) for i in range(0,len(tfidf_all)))
print("Number of Dimensions: {}".format(num_dims))

Number of Dimensions: 2706


That's much too high-dimensional for us to visualize! In order to make it understandable to human eyes, we'll need to reduce dimensionality to 2 or 3 dimensions.  

### Reducing Dimensionality

To do this, we'll use a technique called **_t-SNE_** (short for _t-Stochastic Neighbors Embedding_).  This is too complex for us to code ourselves, so we'll make use of sklearn's implementation of it.  

First, we need to pull the words out of the dictionaries stored in `tf_idf_all_docs` so that only the values remain, and store them in lists instead of dictionaries.  This is because the t-SNE object only works with Array-like objects, not dictionaries.  

In the cell below, create a list of lists that contains a list representation of the values of each of the dictionaries stored in `tf_idf_all_docs`.  The same structure should remain--e.g. the first list should contain only the values that were in the 1st dictionary in `tf_idf_all_docs`, and so on. 

In [84]:
tf_idf_vals_list = []

for i in tf_idf_all_docs:
    tf_idf_vals_list.append(list(i.values()))
    
(tf_idf_vals_list[:2])

[[0.0057839023929447606,
  0.0010874236095693062,
  0.009133330102298753,
  0.007020076503030628,
  0.009133330102298753,
  0.027399990306896257,
  0.002721262820904997,
  0.009133330102298753,
  0.009133330102298753,
  0.0018226737827915257,
  0.0057839023929447606,
  0.007020076503030628,
  0.009133330102298753,
  0.003200677208837432,
  0.007020076503030628,
  0.007020076503030628,
  0.012679521595608754,
  0.009133330102298753,
  0.007020076503030628,
  0.0036706487936766347,
  0.009133330102298753,
  0.004906822903762501,
  0.0052534501962494436,
  0.009133330102298753,
  0.009133330102298753,
  0.009133330102298753,
  0.007020076503030628,
  0.007020076503030628,
  0.007020076503030628,
  0.009133330102298753,
  0.014040153006061255,
  0.009133330102298753,
  0.009133330102298753,
  0.0023457299262599307,
  0.004868949367181534,
  0.009133330102298753,
  0.005587138608988751,
  0.004906822903762501,
  0.0052534501962494436,
  0.0057839023929447606,
  0.0016061054216132066,
  0.00

Now that we have only the values, we can use the `TSNE` object from `sklearn` to transform our data appropriately.  In the cell below, create a `TSNE` with `n_components=3` passed in as a parameter.  Then, use the created object's `fit_transform()` method to transform the data stored in `tf_idf_vals_list` into 3-dimensional data.  Then, inspect the newly transformed data to confirm that it has the correct dimensionality. 

In [83]:

t_sne_object_3d = TSNE(n_components=3)
transformed_data_3d = t_sne_object_3d.fit_transform(tf_idf_vals_list)
transformed_data_3d

ValueError: setting an array element with a sequence.

We'll also want to check out how the visualization looks in 2d.  Repeat the process above, but this time, create a `TSNE` object with 2 components instead of 3.  Again, use `fit_transform()` to transform the data and store it in the variable below, and then inspect it to confirm the transformed data has only 2 dimensions. 

In [67]:

t_sne_object_2d = TSNE(n_components=2)
transformed_data_2d = t_sne_object_2d.fit_transform(tf_idf_vals_list)
transformed_data_2d

ValueError: setting an array element with a sequence.

Now, let's visualize everything!  Run the cell below to a 3D visualization of the songs.

In [ ]:
kendrick_3d = transformed_data_3d[:10]
k3_x = [i[0] for i in kendrick_3d]
k3_y = [i[1] for i in kendrick_3d]
k3_z = [i[2] for i in kendrick_3d]

garth_3d = transformed_data_3d[10:]
g3_x = [i[0] for i in garth_3d]
g3_y = [i[1] for i in garth_3d]
g3_z = [i[2] for i in garth_3d]

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(k3_x, k3_y, k3_z, c='b', s=60, label='Kendrick')
ax.scatter(g3_x, g3_y, g3_z, c='red', s=60, label='Garth')
ax.view_init(30, 10)
ax.legend()
plt.show()

kendrick_2d = transformed_data_2d[:10]
k2_x = [i[0] for i in kendrick_2d]
k2_y = [i[1] for i in kendrick_2d]

garth_2d = transformed_data_2d[10:]
g2_x = [i[0] for i in garth_2d]
g2_y = [i[1] for i in garth_2d]

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(222)
ax.scatter(k2_x, k2_y, c='b', label='Kendrick')
ax.scatter(g2_x, g2_y, c='red', label='Garth')
ax.legend()
plt.show()

Interesting! Take a crack at interpreting these graphs by answering the following question below:

What does each graph mean? Do you find one graph more informative than the other? Do you think that this method shows us discernable differences between Kendrick Lamar songs and Garth Brooks songs?  Use the graphs and your understanding of TF-IDF to support your answer.  

Write your answer to this question below this line:
________________________________________________________________________________________________________________________________

Both graphs show a basic trend among the red and blue dots, although the 3-dimensional graph is more informative than the 2-dimensional graph.  We see a separation between the two artists because they both have words that they use, but the other artist does not.  The words in each song that are common to both are reduced very small numbers or to 0, because of the log operation in the IDF function.  This means that the elements of each song vector with the highest values will be the ones that have words that are unique to that specific document, or at least are rarely used in others.  

## Summary

In this lab, we learned how to: 
* Tokenize a corpus of words and identify the different choices to be made while parsing them
* Use a Count Vectorization strategy to create a Bag of Words
* Use TF-IDF Vectorization with multiple documents to identify words that are important/unique to certain documents
* Visualize and compare vectorized text documents